In [2]:
import pandas as pd
import numpy as np

# Seed for reproducibility
np.random.seed(77)

# Number of drivers
n = 1500

# Options
genders = ["Male", "Female", "Other"]
cities = ["Metro", "Urban", "Town"]

# Create synthetic ride-hailing driver dataset
data = pd.DataFrame({
    "Driver_ID": range(1, n+1),
    "Age": np.random.randint(21, 65, n),
    "Gender": np.random.choice(genders, n, p=[0.6, 0.35, 0.05]),
    "City_Type": np.random.choice(cities, n, p=[0.4, 0.45, 0.15]),
    "Trips_Completed": np.random.randint(50, 2000, n),
    "Avg_Rating": np.round(np.random.uniform(3.0, 5.0, n), 2),  # rating out of 5
    "Complaints": np.random.randint(0, 20, n),
    "Accidents": np.random.randint(0, 5, n)
})

# Step 1: Base rule — complaints or accidents increase review likelihood
condition = (data["Complaints"] > 5) | (data["Accidents"] > 0)
data.loc[condition, "Flagged_For_Review"] = np.random.choice([1, 0], condition.sum(), p=[0.7, 0.3])
data.loc[~condition, "Flagged_For_Review"] = np.random.choice([1, 0], (~condition).sum(), p=[0.15, 0.85])

# Step 2: Extra bias — Metro drivers are more likely to be flagged
mask_metro = data["City_Type"] == "Metro"
data.loc[mask_metro, "Flagged_For_Review"] = np.random.choice([1, 0], mask_metro.sum(), p=[0.4, 0.6])

# Preview dataset
print(data.head())


   Driver_ID  Age  Gender City_Type  Trips_Completed  Avg_Rating  Complaints  \
0          1   44   Other     Metro              324        3.88          12   
1          2   52    Male     Urban              323        3.96          17   
2          3   41  Female     Urban              448        3.53           4   
3          4   41  Female      Town             1485        4.83          19   
4          5   64    Male     Urban              501        3.66          15   

   Accidents  Flagged_For_Review  
0          3                 1.0  
1          3                 0.0  
2          4                 1.0  
3          1                 0.0  
4          2                 0.0  


In [3]:
# =========================================
# Task 1: Bias Assessment Before Deployment
# =========================================
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Prepare data
X = pd.get_dummies(data.drop(columns=["Driver_ID", "Flagged_For_Review"]), drop_first=True)
y = data["Flagged_For_Review"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Train baseline model
model = RandomForestClassifier(n_estimators=50, random_state=42)
model.fit(X_train, y_train)

# Evaluate
y_pred = model.predict(X_test)
print("Baseline Accuracy:", accuracy_score(y_test, y_pred))

# Attach sensitive attributes back
df_test = X_test.copy()
df_test["y_pred"] = y_pred
df_test["Gender"] = data.loc[y_test.index, "Gender"]
df_test["City_Type"] = data.loc[y_test.index, "City_Type"]

# Positive prediction rates
gender_rates = df_test.groupby("Gender")["y_pred"].mean()
city_rates = df_test.groupby("City_Type")["y_pred"].mean()

print("\nPositive prediction rate by Gender:\n", gender_rates)
print("\nPositive prediction rate by City_Type:\n", city_rates)


Baseline Accuracy: 0.5955555555555555

Positive prediction rate by Gender:
 Gender
Female    0.510638
Male      0.562500
Other     0.857143
Name: y_pred, dtype: float64

Positive prediction rate by City_Type:
 City_Type
Metro    0.186170
Town     0.753425
Urban    0.857143
Name: y_pred, dtype: float64


In [4]:
# =========================================
# Task 2: Ethics-Aware Fairness Gate
# =========================================
def fairness_check(df, sensitive_attr, threshold=0.08):
    rates = df.groupby(sensitive_attr)["y_pred"].mean()
    disparity = rates.max() - rates.min()
    print(f"{sensitive_attr} disparity: {disparity:.3f}")
    return disparity <= threshold

if not fairness_check(df_test, "Gender"):
    print("❌ Fairness check failed for Gender.")
else:
    print("✅ Fairness check passed for Gender.")

if not fairness_check(df_test, "City_Type"):
    print("❌ Fairness check failed for City_Type.")
else:
    print("✅ Fairness check passed for City_Type.")


Gender disparity: 0.347
❌ Fairness check failed for Gender.
City_Type disparity: 0.671
❌ Fairness check failed for City_Type.


In [5]:
# =========================================
# Task 3: Feature Importance Ethics Audit
# =========================================
importances = pd.Series(model.feature_importances_, index=X.columns).sort_values(ascending=False)
print(importances.head(10))

# Flag socio-demographic features
socio_features = ["Gender_Male", "Gender_Other", "City_Type_Town", "City_Type_Urban"]
for feat in socio_features:
    if feat in importances.head(10).index:
        print(f"⚠️ Feature '{feat}' ranks high — consider mitigation.")


Trips_Completed    0.231122
Avg_Rating         0.212556
Age                0.185937
Complaints         0.156722
Accidents          0.077258
City_Type_Urban    0.065511
Gender_Male        0.031020
City_Type_Town     0.026821
Gender_Other       0.013054
dtype: float64
⚠️ Feature 'Gender_Male' ranks high — consider mitigation.
⚠️ Feature 'Gender_Other' ranks high — consider mitigation.
⚠️ Feature 'City_Type_Town' ranks high — consider mitigation.
⚠️ Feature 'City_Type_Urban' ranks high — consider mitigation.


In [6]:
# =========================================
# Task 4: Ethical Development Documentation
# =========================================
user_stories = [
    "As a safety compliance officer, I want the system to evaluate drivers without bias toward city type so that all drivers are judged fairly.",
    "As an operations manager, I want the system to provide clear reasons when flagging a driver, so drivers understand and can respond."
]

for story in user_stories:
    print("-", story)


- As a safety compliance officer, I want the system to evaluate drivers without bias toward city type so that all drivers are judged fairly.
- As an operations manager, I want the system to provide clear reasons when flagging a driver, so drivers understand and can respond.


In [7]:
# =========================================
# Task 5: Accountability via Prediction Logging
# =========================================
import datetime

MODEL_VERSION = "v1.0"
prediction_log = []

def predict_and_log(driver_data):
    pred = int(model.predict(driver_data)[0])
    log_entry = {
        "timestamp": datetime.datetime.now().isoformat(),
        "model_version": MODEL_VERSION,
        "inputs": driver_data.to_dict(orient="records")[0],
        "prediction": pred
    }
    prediction_log.append(log_entry)
    return pred

# Example
sample_driver = X_test.iloc[[0]]
predict_and_log(sample_driver)
prediction_log[:2]


[{'timestamp': '2025-08-13T10:45:40.158486',
  'model_version': 'v1.0',
  'inputs': {'Age': 56,
   'Trips_Completed': 1167,
   'Avg_Rating': 3.95,
   'Complaints': 9,
   'Accidents': 3,
   'Gender_Male': True,
   'Gender_Other': False,
   'City_Type_Town': False,
   'City_Type_Urban': True},
  'prediction': 1}]

In [8]:
# =========================================
# Task 6: Post-Deployment Monitoring
# =========================================
gender_disparity = gender_rates.max() - gender_rates.min()
city_disparity = city_rates.max() - city_rates.min()

if gender_disparity > 0.08:
    print("⚠️ Alert: Gender fairness threshold exceeded.")
else:
    print("✅ Gender fairness acceptable.")

if city_disparity > 0.08:
    print("⚠️ Alert: City fairness threshold exceeded.")
else:
    print("✅ City fairness acceptable.")


⚠️ Alert: Gender fairness threshold exceeded.
⚠️ Alert: City fairness threshold exceeded.


In [9]:
# =========================================
# Task 7: Privacy Protection
# =========================================
import hashlib

data["Driver_ID"] = data["Driver_ID"].apply(lambda x: hashlib.sha256(str(x).encode()).hexdigest())
data.head()


,Driver_ID,Age,Gender,City_Type,Trips_Completed,Avg_Rating,Complaints,Accidents,Flagged_For_Review
0,6b86b273ff34fce19d6b804eff5a3f5747ada4eaa22f1d...,44,Other,Metro,324,3.88,12,3,1.0
1,d4735e3a265e16eee03f59718b9b5d03019c07d8b6c51f...,52,Male,Urban,323,3.96,17,3,0.0
2,4e07408562bedb8b60ce05c1decfe3ad16b72230967de0...,41,Female,Urban,448,3.53,4,4,1.0
3,4b227777d4dd1fc61c6f884f48641d02b4d121d3fd328c...,41,Female,Town,1485,4.83,19,1,0.0
4,ef2d127de37b942baad06145e54b0c619a1f22327b2ebb...,64,Male,Urban,501,3.66,15,2,0.0


In [10]:
# =========================================
# Task 8: Inclusivity Feedback Loop
# =========================================
feedback = pd.DataFrame({
    "Driver_ID": ["new_1", "new_2"],
    "Age": [34, 42],
    "Gender": ["Other", "Other"],
    "City_Type": ["Town", "Town"],
    "Trips_Completed": [500, 750],
    "Avg_Rating": [4.8, 4.6],
    "Complaints": [0, 1],
    "Accidents": [0, 0],
    "Flagged_For_Review": [0, 0]
})

data_updated = pd.concat([data, feedback], ignore_index=True)
print("Updated dataset size:", data_updated.shape)


Updated dataset size: (1502, 9)


In [11]:
# =========================================
# Task 9: Incident Response Simulation
# =========================================
incident_report = {
    "Detection": "Monitoring shows Metro drivers are flagged at double the rate of other cities.",
    "Investigation": "Analyze model outputs and input features; check for correlations with city type.",
    "Communication": "Notify compliance, legal, and driver relations teams; prepare public communication plan.",
    "Remediation": "Retrain model with balanced data; reduce or remove city type influence."
}

for step, detail in incident_report.items():
    print(f"{step}: {detail}")


Detection: Monitoring shows Metro drivers are flagged at double the rate of other cities.
Investigation: Analyze model outputs and input features; check for correlations with city type.
Communication: Notify compliance, legal, and driver relations teams; prepare public communication plan.
Remediation: Retrain model with balanced data; reduce or remove city type influence.
